## 2.9. StackOverflow Pages

* `LIMIT` 변수 필요하지 않음.
* 절차
    1. get the page
    2. request 요청 보내기
    3. job 추출
    4. 
    
* 함수 만드는 방법

[so.py]
    
1. 최종 함수 먼저 써 놓는다. 반환할 결과물은 임시로 빈 리스트를 생성한다.
    ```
    def get_jobs():
        return []
    ```
    
2. 절차에 따라 진행해야 할 작업을 최종 함수 안에 적고, 위쪽에 그 함수들을 적는다.
    ```
    def get_last_page():
        ...
        return
        
    def extract_jobs(last_page):
        ...
        return jobs
    
    def get_jobs():
        last_page = get_last_page()
        jobs = extract_jobs(last_page)
        ...
        return []
    ```
    
[main.py]
1. 함수 import
    ```
    from so import get_jobs
    ```
        

* pagination 태그 구조 변경
* Q) 왜 계속 4페이지까지밖에 안 나올까?

In [7]:
# module import
import requests
from bs4 import BeautifulSoup

# 변수
URL = f"https://stackoverflow.com/jobs?q=python&sort=i"

# 마지막 페이지 반환
def get_last_page():
    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'lxml')
    pages = soup.find("div", {"class" : "s-pagination"}).find_all("a")# 태그 변경
    print(pages)

# 직업 추출
def get_jobs():
    last_page = get_last_page()
    return []
    

# main.py
so_jobs = get_jobs()
print(so_jobs)

[<a class="s-pagination--item is-selected" href="/jobs?q=python&amp;sort=i" title="page 1 of 162">
<span>1</span>
</a>, <a class="s-pagination--item" href="/jobs?q=python&amp;sort=i&amp;pg=2" title="page 2 of 162">
<span>2</span>
</a>, <a class="s-pagination--item" href="/jobs?q=python&amp;sort=i&amp;pg=3" title="page 3 of 162">
<span>3</span>
</a>, <a class="s-pagination--item" href="/jobs?q=python&amp;sort=i&amp;pg=4" title="page 4 of 162">
<span>4</span>
</a>, <a class="s-pagination--item" href="/jobs?q=python&amp;sort=i&amp;pg=162" title="page 162 of 162">
<span>162</span>
</a>, <a class="s-pagination--item" href="/jobs?q=python&amp;sort=i&amp;pg=2" title="page 2 of 162">
<span>next</span><i class="material-icons">chevron_right</i>
</a>]
[]


## 2.10. StackOverflow extract_jobs

* 문제 : 내가 계속 이상하다고 생각했던 pagination 문제와 같은 문제.
    - 그냥 162가 마지막인가보다 하고 넘어감.
* `get_text()` 메서드 : `strip=True` 옵션 있음.

In [24]:
# 마지막 페이지 숫자로 반환
def get_last_page():
    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'lxml')
    pages = soup.find("div", {"class" : "s-pagination"}).find_all("a")
    last_page = pages[-2].get_text(strip=True) # page[-1] : 마지막 Next, page[-2] : 그 이전 마지막 페이지
    return int(last_page)

def extract_jobs(last_page):
    jobs = []
    for page in range(last_page):
        print(f"Scrapping page {page+1}")
        req = requests.get(f"{URL}&pg={page+1}")
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'lxml')
        results = soup.find_all("div", {"class":"grid"})
        for result in results:
#         title = soup.find_all("h2")
#     for t in title:
#         print(t.get_text(strip=True))
    
    return jobs

def get_jobs():
    last_page = get_last_page()
    jobs = extract_jobs(last_page)
    return []

get_jobs()

Scrapping page 1
200
Scrapping page 2
200
Scrapping page 3
200
Scrapping page 4
200
Scrapping page 5
200
Scrapping page 6
200
Scrapping page 7
200
Scrapping page 8
200
Scrapping page 9
200
Scrapping page 10
200
Scrapping page 11
200
Scrapping page 12
200
Scrapping page 13
200
Scrapping page 14
200
Scrapping page 15
200
Scrapping page 16
200
Scrapping page 17
200
Scrapping page 18
200
Scrapping page 19
200
Scrapping page 20
200
Scrapping page 21
200
Scrapping page 22
200
Scrapping page 23
200
Scrapping page 24
200
Scrapping page 25
200
Scrapping page 26
200
Scrapping page 27
200
Scrapping page 28
200
Scrapping page 29
200
Scrapping page 30
200
Scrapping page 31
200
Scrapping page 32
200
Scrapping page 33
200
Scrapping page 34
200
Scrapping page 35
200
Scrapping page 36
200
Scrapping page 37
200
Scrapping page 38
200
Scrapping page 39
200
Scrapping page 40
200
Scrapping page 41
200
Scrapping page 42
200
Scrapping page 43
200
Scrapping page 44
200
Scrapping page 45
200
Scrapping page 46
2

KeyboardInterrupt: 

## 2.11. StackOverflow extract_job 2

* 직무명 나타내는 태그 달라짐
* 회사 이름 + 위치 나타내는 태그 같이 있음.
    - `recursive = False` 옵션 : 첫 단계에 있는 요소만 가져옴.
    - unpacking values : 리스트에 두 가지 요소가 있다는 것을 알고, 내가 그 요소가 무엇을 의미하는지 알기 때문에 사용할 수 있음.

In [62]:
def get_last_page():
    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'lxml')
    pages = soup.find("div", {"class" : "s-pagination"}).find_all("a")
    last_page = pages[-2].get_text(strip=True) # page[-1] : 마지막 Next, page[-2] : 그 이전 마지막 페이지
    return int(last_page)

def extract_job(html): # BeautifulSoup 객체를 인자로 받음.
    # title = html.find("div", {"class" :"grid--cell fl1"}).find("h2").find("a")["title"]
    company, location = html.find("div", {"class" : "grid--cell fl1"}).find("h3").find_all("span", recursive=False)
    print(company.get_text(strip=True), location.get_text(strip=True))
    
#content > div.js-search-container.search-container.mbn24 > div.grid.fd-row.flush-left > div.grid--cell.fl1.br.bc-black-2 > div > div.listResults > div:nth-child(3) > div:nth-child(2) > div.grid--cell.fl1
def extract_jobs(last_page):
    jobs = []
    for page in range(last_page):
        print(f"Scrapping page {page+1}")
        req = requests.get(f"{URL}&pg={page+1}")
        soup = BeautifulSoup(req.text, 'lxml')
        results = soup.find_all("div", {"class":"-job"})
        for result in results:
            job = extract_job(result) # 위의 함수로 result 넘김.
            jobs.append(job)   
    return jobs

def get_jobs():
    last_page = get_last_page()
    jobs = extract_jobs(last_page)
    return []

get_jobs()

Scrapping page 1
READY Robotics Columbus, OH
Facebook Tel Aviv-Yafo, Israel
Facebook Fremont, CA
Facebook Los Angeles, CA
Facebook Menlo Park, CA
Facebook London, UK
Facebook Tel Aviv-Yafo, Israel
Facebook New York, NY
Facebook Menlo Park, CA
Facebook Menlo Park, CA
Macquarie BankviaHays plc London, UK
Marex Spectron London, UK
BCG Gamma Stockholm, Sweden
Camlin Group Parma, Italy
BCG Gamma Munich, Germany
BCG Gamma London, UK
Camlin Group Bari, Italy
BCG Gamma Paris, France
PeopleFun Richardson, TX
Leverege Rockville, MD
JC Development Glarus, Switzerland
TPP Horsforth, UK
ML6 Ghent, Belgium
ARTIDIS AG Basel, Switzerland
Remedy Analytics Milwaukee, WI
Scrapping page 2
Siemens AG Amadora, Portugal
IOX GmbH Düsseldorf, Deutschland
Resource Systems Group Arlington, VA
relayr GmbH Katowice, Polen
Solidus Labs Tel Aviv-Yafo, Israel
Solidus Labs New York, NY
Solidus Labs Tel Aviv-Yafo, Israel
BCG Gamma Warsaw, Poland
BCG Gamma Helsinki, Finland
BCG Gamma Stockholm, Sweden
BCG Gamma Munich, 

Retty株式会社 Minato City, Japan
株式会社Donuts Shibuya City, Japan
株式会社 ハートビーツ Shinjuku City, Japan
BizMobile株式会社 Chiyoda City, Japan
株式会社Gunosy Minato City, Japan
株式会社レアジョブ Shibuya City, Japan
株式会社レアジョブ Shibuya City, Japan
株式会社MonotaRO Minato City, Japan
株式会社 アカリ Shibuya City, Japan
株式会社 TORICO Chiyoda City, Japan
株式会社ダイヤモンドファンタジー Chuo City, Japan
株式会社パソナテック Chiyoda City, Japan
Fabeee 株式会社 Chiyoda City, Japan
株式会社ダイヤモンドファンタジー Chuo City, Japan
株式会社ダイヤモンドファンタジー Chuo City, Japan
株式会社ダイヤモンドファンタジー Chuo City, Japan
株式会社Donuts Shibuya City, Japan
Fabeee 株式会社 Chiyoda City, Japan
Fabeee 株式会社 Chiyoda City, Japan
シェルフィー 株式会社 Shibuya City, Japan
フェアリーデバイセズ 株式会社 Bunkyo City, Japan
株式会社エーティーエス Minato City, Japan
株式会社スタッフサービス Hamamatsu, Japan
株式会社アーティス Hamamatsu, Japan
株式会社スタッフサービス　エンジニアガイド Hamamatsu, Japan
Scrapping page 13
株式会社スタッフサービス　エンジニアガイド Mishima, Japan
株式会社ビーネックステクノロジーズ Nagoya, Japan
株式会社アドキットインフォケーション Chiyoda City, Japan
株式会社 オプティマインド Nagoya, Japan
株式会社 オプティマインド Nagoya, Japan
株式会社ビーネックステクノロジーズ Nu

Apple New York, NY
Apple Cupertino, CA
Apple Cupertino, CA
Apple Cupertino, CA
Apple Culver City, CA
Apple Cupertino, CA
Apple Cupertino, CA
Apple Cupertino, CA
Apple Cupertino, CA
Jumbo Supermarkets Veghel, Netherlands
QuantumBlack London, UK
Adevinta Paris, France
Nationwide Building Society London, UK
BeCode VZW Antwerpen, Belgium
Delivery Hero SE Berlin, Germany
leboncoin Paris, France
trivago N.V. Düsseldorf, Germany
Envelop Risk Bristol, UK
envelio GmbH Köln, Deutschland
envelio GmbH Köln, Deutschland
European Bioinformatics Institute (EMBL-EBI) Hinxton, UK
European Bioinformatics Institute (EMBL-EBI) Hinxton, UK
X-Team No office location
European Bioinformatics Institute (EMBL-EBI) Hinxton, Vereinigtes Königreich
Zoopla London, UK
Scrapping page 24
brainbot Berlin, Germany
Verimi GmbH Berlin, Germany
Vistaprint Barcelona, Spain
QuantumBlack London, UK
Picnic Amsterdam, Netherlands
SG Digital Bristol, UK
Autify, Inc. Tokyo, Japan
SkillValue No office location
jambit GmbH München,

株式会社アウスタ Saitama, Japan
株式会社アウスタ Saitama, Japan
株式会社エーティーエス Shinjuku City, Japan
株式会社ブレーンネット Koto City, Japan
株式会社スタッフサービス Minato City, Japan
エヌ次元株式会社 Shibuya City, Japan
リブライト株式会社 Ueda, Japan
リブライト株式会社 Ueda, Japan
デジタル・アドバタイジング・コンソーシアム株式会社 Shibuya City, Japan
エムスリー株式会社 Minato City, Japan
株式会社KADOKAWA Connected Chuo City, Japan
株式会社KADOKAWA Connected Chuo City, Japan
Tech Fun 株式会社 Taito City, Japan
株式会社KADOKAWA Connected Chuo City, Japan
株式会社C-mind Shinjuku City, Japan
デジタル・アドバタイジング・コンソーシアム株式会社 Shibuya City, Japan
株式会社 テレビ朝日メディアプレックス Minato City, Japan
JP Morgan Chase Bengaluru, India
Unnati Bengaluru, India
Insights Software Solutions Bengaluru, India
Stratmark Consulting Bengaluru, India
CIEL HR Services Bengaluru, India
Bizita Technologies Private Limited Bengaluru, India
Verizon Hyderabad, India
Wells Fargo Hyderabad, India
Scrapping page 35
TGS Irvine, CA
SBヒューマンキャピタル株式会社 Shinjuku City, Japan
Financepeer Hyderabad, India
Triangle Global Pune, India
mquotient Pune, India
PeopleNort

アイレット株式会社 Osaka, Japan
アデコ株式会社 Suita, Japan
レールコンサルティング株式会社 Chuo City, Japan
株式会社 ソニックジャム Minato City, Japan
有限会社 ファンキーコープ Minato City, Japan
SBテクノロジー株式会社 Minato City, Japan
株式会社SIGNATE Chiyoda City, Japan
株式会社スタッフサービス Sayama, Japan
株式会社Her's Kobe, Japan
株式会社リネア Minato City, Japan
株式会社クロス・マーケティンググループ Shinjuku City, Japan
株式会社エルバーラ・スイス Shinagawa City, Japan
株式会社 Sun Asterisk Chiyoda City, Japan
株式会社ZOZOテクノロジーズ Minato City, Japan
type転職エージェント Shibuya City, Japan
ログリー株式会社 Shibuya City, Japan
LINE株式会社 Shinjuku City, Japan
株式会社ウェルネストコミュニケーションズ Shibuya City, Japan
株式会社with Shibuya City, Japan
インディゴ株式会社 Setagaya City, Japan
株式会社Holmes Chiyoda City, Japan
株式会社ZOZOテクノロジーズ Minato City, Japan
株式会社ZOZOテクノロジーズ Minato City, Japan
株式会社iforward Shibuya City, Japan
株式会社ZOZOテクノロジーズ Minato City, Japan
Scrapping page 45
GLナビゲーション株式会社 Toshima City, Japan
株式会社ZOZOテクノロジーズ Minato City, Japan
株式会社セルシス Shinjuku City, Japan
ログリー株式会社 Shibuya City, Japan
株式会社エルバーラ・スイス Shinagawa City, Japan
株式会社よきあす Chuo City, Japa

...

Walgreens Deerfield, IL
Octave Group Montreal, QC, Canada
Polarsteps Amsterdam, Netherlands
Vital Images Minnetonka, MN
Bloomberg LP San Francisco, CA
Bloomberg LP New York, NY
Bloomberg LP London, UK
Bloomberg LP New York, NY
buynomics GmbH Köln, Deutschland
Flow Traders Amsterdam, Netherlands
Goldman Sachs Stockholm, Sweden
M-Way Solutions GmbH Stuttgart, Deutschland
Harmonia Holdings Group, LLC Charlottesville, VA
Paradox Security Systems Saint-Eustache, QC, Canada
iTranslate GmbH Graz, Austria
Daugherty Business Solutions Creve Coeur, MO
Epidemic Sound AB Stockholm, Sweden
Delivery Hero SE Berlin, Germany
McKinsey & Company Warsaw, Poland
ClimatePartner München, Deutschland
RealVNC Cambridge, UK
Mirakl Paris, France
DSW Zorgverzekeraar Schiedam, Netherlands
Blockchain Consulting GmbH München, Deutschland
SG Digital Kraków, Poland
Scrapping page 128
General MotorsviaPandoLogic Warren, MI
Citizens Advice London, UK
Leica Geosystems AG Heerbrugg, Schweiz
CashShield Singapore
EmbodyMe 

Varen Technologies Columbia, MD
Ignitarium Technology Solutions Bengaluru, India
Bidgely Bengaluru, India
Inkers Bengaluru, India
Pinnacle Consultz Bengaluru, India
CrunchMetrics Bengaluru, India
Careerera Bengaluru, India
Magnusson Institute Reno, NV
Gorgias Paris, France
Rollbar Barcelona, Spain
Density No office location
Density No office location
Nucleome Therapeutics Oxford, UK
Nucleome Therapeutics Oxford, UK
Rollbar Budapest, Hungary
Captivation Software Annapolis Junction, MD
Tempo Software Montreal, QC, Canada
Empirico Inc. San Diego, CA
Jobscan Seattle, WA
Magnusson Institute Reno, NV
MayStreet Inc. New York, NY
Pivotal Energy Solutions No office location
Varen Technologies Columbia, MD
Interactions New Providence, NJ
StealthPath Reston, VA
Scrapping page 138
TravelPerk Barcelona, Spain
StealthPath Reston, VA
User Research International Redmond, WA
American Water Camden, NJ
Carbon Relay Washington, DC
Carbon Relay Washington, DC
Xactware Lexington, NE
Exosite Minneapolis, MN


Mobiquity Europe Amsterdam, Netherlands
Aktis Hlavní město Praha, Czechia
Aktis Hlavní město Praha, Czechia
IAV Gifhorn, Deutschland
Smart Reporting GmbH München, Germany
Facebook Menlo Park, CA
Facebook Boston, MA
Facebook Bellevue, WA
Facebook Los Angeles, CA
Facebook Menlo Park, CA
Facebook Tel Aviv-Yafo, Israel
Facebook Menlo Park, CA
Facebook Menlo Park, CA
Look4IT Domaradzki Sp. J Warsaw, Poland
Timme Hosting GmbH & Co. KG Lüneburg, Deutschland
Derivco Barcelona, Spain
finleap GmbH Berlin, Germany
株式会社ビーネックステクノロジーズ Kawasaki, Japan
株式会社エーティーエス Yokohama, Japan
Hamee 株式会社 Odawara, Japan
株式会社マクニカ Yokohama, Japan
株式会社スタッフサービス Yokohama, Japan
株式会社エーティーエス Sagamihara, Japan
株式会社 クリティックミッションジャパン Yokohama, Japan
株式会社スタッフサービス Kawasaki, Japan
Scrapping page 149
Odoo Ramillies, Belgium
株式会社 テコテック Minato City, Japan
株式会社Donuts Shibuya City, Japan
RAYLYTIC GmbH Leipzig, Deutschland
株式会社エーティーエス Shibuya City, Japan
LINE株式会社 Shinjuku City, Japan
LINE株式会社 Fukuoka, Japan
LINE株式会社 Fukuoka, Japan
LINE

クラウドエース 株式会社 Fukuoka, Japan
株式会社 リンネット Fukuoka, Japan
木村情報技術 株式会社 Fukuoka, Japan
株式会社 リンネット Fukuoka, Japan
コネヒト 株式会社 Minato City, Japan
株式会社 スリーシェイク Chiyoda City, Japan
弁護士ドットコム 株式会社 Minato City, Japan
株式会社 Blue Star Shinagawa City, Japan
CCC マーケティング 株式会社 Shibuya City, Japan
株式会社Photosynth Minato City, Japan
株式会社 Relic Shibuya City, Japan
株式会社 メイキップ Shinjuku City, Japan
株式会社 Blue Star Shinagawa City, Japan
株式会社 ブイ・クルーズ Kyoto, Japan
株式会社 フィーリスト Kyoto, Japan
株式会社 トップゲート Osaka, Japan
株式会社 トップゲート Osaka, Japan
株式会社 PLUS Sapporo, Japan
ネオス 株式会社 Sapporo, Japan
株式会社 スタイル・フリー Nagoya, Japan
株式会社スタッフサービス Hiroshima, Japan
株式会社スタッフサービス Fukuoka, Japan
株式会社 フロムスクラッチ Fukuoka, Japan
株式会社　ジェイテック Fukuoka, Japan
株式会社　ジェイテック Fukuoka, Japan
Scrapping page 160
株式会社　ジェイテック Fukuoka, Japan
株式会社メンバーズエッジ Fukuoka, Japan
富士インフォックス・ネット株式会社 Fukuoka, Japan
株式会社 TORICO Chiyoda City, Japan
株式会社クルージス Chiyoda City, Japan
有限会社 クレスト Osaka, Japan
株式会社スマートバリュー Osaka, Japan
有限会社 クレスト Osaka, Japan
株式会社 アビリブ(旧名:株式会社 アビリティコンサルタント

[]

## 2.12. StackOverflow extract_job 2

In [65]:
def get_last_page():
    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    pages = soup.find("div", {"class" : "s-pagination"}).find_all("a")
    last_page = pages[-2].get_text(strip=True) # page[-1] : 마지막 Next, page[-2] : 그 이전 마지막 페이지
    return int(last_page)

def extract_job(html): 
    title = html.find("div", {"class" :"grid--cell fl1"}).find("h2").find("a")["title"]
    company, location = html.find("div", {"class" : "grid--cell fl1"}).find("h3").find_all("span", recursive=False)
    company = company.get_text(strip=True)
    location = location.get_text(strip=True).strip("-")
    print(location)
    return {'title' : title, 'company' : company, 'location' : location}

def extract_jobs(last_page):
    jobs = []
    for page in range(last_page):
        print(f"Scrapping page {page+1}")
        req = requests.get(f"{URL}&pg={page+1}")
        soup = BeautifulSoup(req.text, 'lxml')
        results = soup.find_all("div", {"class":"-job"})
        for result in results:
            job = extract_job(result) # 위의 함수로 result 넘김.
            jobs.append(job)   
    return jobs

def get_jobs():
    last_page = get_last_page()
    jobs = extract_jobs(last_page)
    return []

get_jobs()

Scrapping page 1
Columbus, OH
Rockville, MD
Los Angeles, CA
Munich, Germany
Tel Aviv-Yafo, Israel
Fremont, CA
Los Angeles, CA
Menlo Park, CA
London, UK
Tel Aviv-Yafo, Israel
New York, NY
Menlo Park, CA
Menlo Park, CA
London, UK
London, UK
Stockholm, Sweden
Parma, Italy
Munich, Germany
London, UK
Bari, Italy
Paris, France
Richardson, TX
Rockville, MD
Glarus, Switzerland
Horsforth, UK
Scrapping page 2
Milwaukee, WI
Ghent, Belgium
Basel, Switzerland
Düsseldorf, Deutschland
Arlington, VA
Katowice, Polen
Tel Aviv-Yafo, Israel
New York, NY
Tel Aviv-Yafo, Israel
Warsaw, Poland
Helsinki, Finland
Stockholm, Sweden
Munich, Germany
London, UK
Paris, France
London, UK
Berlin, Germany
Boston, MA
Marousi, Greece
Rotterdam, Netherlands
Bracknell, UK
Barcelona, Spain
Amadora, Portugal
Cluj-Napoca, Romania
İstanbul, Turkey
Scrapping page 3
Berlin, Deutschland
Prague, Czechia
Bengaluru, India
Cypress, CA
Edinburgh, UK
Cologne, Germany
New York, NY
Singapore
Singapore
Singapore
München, Germany
Singapore

Glasgow, UK
No office location
Vila Nova de Gaia, Portugal
Mountain View, CA
Mountain View, CA
Mountain View, CA
Mountain View, CA
Vila Nova de Gaia, Portugal
Paris, France
München, Deutschland
Austin, TX
Bath, UK
Halifax, NS, Canada
Milano, Italy
München, Deutschland
Amsterdam, Netherlands
No office location
No office location
No office location
No office location
Little Rock, AR
Cambridge, MA
Berlin, Deutschland
Charleston, SC
Montreal, QC, Canada
Scrapping page 21
Cupertino, CA
Berlin, Deutschland
Munich, Germany
Modena, Italy
Dallas, TX
Austin, TX
Cupertino, CA
Cupertino, CA
Cupertino, CA
Austin, TX
Cupertino, CA
Seattle, WA
Austin, TX
Cupertino, CA
Cupertino, CA
Cupertino, CA
Cupertino, CA
Shanghai, China
Singapore
Shanghai, China
London, UK
Cambridge, UK
Berlin, Germany
Hyderabad, India
Herzliya, Israel
Scrapping page 22
Cupertino, CA
Cupertino, CA
Cupertino, CA
Cupertino, CA
Cupertino, CA
Modena, Italy
Charleston, SC
Berlin, Germany
Berlin, Germany
No office location
München, Ge

San Francisco, CA
Tampa, FL
Columbus, OH
Columbus, OH
Austin, TX
New York, NY
New York, NY
Karlsruhe, Deutschland
Karlsruhe, Deutschland
Karlsruhe, Deutschland
San Francisco, CA
Chicago, IL
Tampa, FL
San Francisco, CA
Colorado Springs, CO
Austin, TX
Chicago, IL
Dublin, Ireland
Redmond, WA
Columbus, OH
No office location
Oxford, UK
London, UK
London, UK
The Hague, Netherlands
Scrapping page 41
Jersey City, NJ
Berlin, Germany
Dortmund, Deutschland
Jersey City, NJ
No office location
Jersey City, NJ
San Francisco, CA
New York, NY
Leipzig, Deutschland
Berlin, Deutschland
Espoo, Finland
Köln, Deutschland
Ottignies-Louvain-la-Neuve, Belgium
London, UK
Coventry, UK
Amsterdam, Netherlands
Berlin, Germany
Berlin, Germany
Stuttgart, Deutschland
Zürich, Switzerland
Stockholm, Sweden
Shanghai, China
Stuttgart, Deutschland
Stuttgart, Deutschland
No office location
Scrapping page 42
Schwarzheide, Germany
Berlin, Deutschland
Abu Dhabi, United Arab Emirates
Hyderabad, India
Hyderabad, India
Hyderabad, 

Dubai, United Arab Emirates
Amsterdam, Netherlands
Amsterdam, Netherlands
Amsterdam, Netherlands
Amsterdam, Netherlands
Chiasso, Switzerland
Milford Charter Township, MI
Warren, MI
Hannover, Deutschland
Stuttgart, Deutschland
Milano, Italy
München, Deutschland
Hyderabad, India
Shinjuku City, Japan
Berlin, Germany
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Scrapping page 60
Rotterdam, Netherlands
Hyderabad, India
Hyderabad, India
Hyderabad, India
Hyderabad, India
Berlin, Germany
Hyderabad, India
New Delhi, India
Los Angeles, CA
Pune, India
Pune, India
Shinjuku City, Japan
Shibuya City, Japan
Minato City, Japan
Chiyoda City, Japan
Minato City, Japan
Warren, MI
Rotterdam, Netherlands
Rotterdam, Niederlande
Chiyoda City, Japan
Tsurugashima-shi, Japan
Chiyoda City, Japan
New York, NY
Minato City, Japan
Minato City, Japan
Scrapping page 61
Kofu, Japan
Shibuya City, J

Herzliya, Israel
Amsterdam, Netherlands
London, UK
London, UK
No office location
Amsterdam, Netherlands
Biberach, Germany
Berlin, Germany
Berlin, Germany
München, Deutschland
No office location
Helsinki, Finland
München, Germany
Menlo Park, CA
Teheran-ro 108-gil, South Korea
Kawasaki, Japan
Shibuya City, Japan
Göteborg, Sweden
Yokohama, Japan
Koto City, Japan
Chuo City, Japan
Chiyoda City, Japan
Numazu, Japan
Setagaya City, Japan
Minato City, Japan
Scrapping page 79
Chicago, IL
Minato City, Japan
Chiyoda City, Japan
Minato City, Japan
San Mateo, CA
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Kolkata, India
Kolkata, India
Baltimore, MD
Chicago, IL
New York, NY
New York, NY
Chicago, IL
Chicago, IL
Chicago, IL
Scrapping page 80
Wilmington, DE
Washington, DC
Cambridge, MA
Geneva, Switzerland
San Francisco, CA
No office location
Mahwah, NJ
Mahwah, NJ

No office location
San Francisco, CA
Boston, MA
Indianapolis, IN
Seattle, WA
San Francisco, CA
Boston, MA
Indianapolis, IN
Boston, MA
Seattle, WA
San Francisco, CA
Seattle, WA
Waterloo, ON, Canada
Petah Tikva, Israel
Hyderabad, India
Bloomington, IN
Tel Aviv-Yafo, Israel
Burlingame, CA
Seattle, WA
Chiasso, Switzerland
New York, NY
Reichenbach im Kandertal, Schweiz
Solana Beach, CA
Solana Beach, CA
London, UK
Scrapping page 98
London, UK
Berlin, Germany
Berlin, Germany
New York, NY
Frankfurt am Main, Germany
Leipzig, Deutschland
No office location
San Francisco, CA
Bellevue, WA
Bengaluru, India
San Francisco, CA
Gdańsk, Poland
Newcastle upon Tyne, UK
Montevideo, Uruguay
Leipzig, Deutschland
Hägersten, Sweden
Karlsruhe, Germany
Nürnberg, Deutschland
München, Germany
Turin, Italy
Freiburg im Breisgau, Deutschland
Boston, MA
Boston, MA
Setagaya City, Japan
Manchester, UK
Scrapping page 99
Courbevoie, France
Dublin, Ireland
München, Germany
Berlin, Deutschland
Courbevoie, France
Frankfurt a

No office location
Lehi, UT
Glendale, CA
Glendale, CA
Malmö, Sweden
Seattle, WA
Portland, OR
No office location
Amsterdam, Netherlands
Medellín, Colombia
Grand Rapids, MI
Hamburg, Deutschland
No office location
No office location
No office location
No office location
New York, NY
Tolochenaz, Switzerland
Baltimore, MD
Baltimore, MD
Providence, RI
Eschborn, Deutschland
Copenhagen, Denmark
Paris, France
Zürich, Switzerland
Scrapping page 117
Hamburg, Deutschland
Sofia, Bulgaria
Bristol, UK
Hamburg, Germany
München, Germany
Stuttgart, Deutschland
Stuttgart, Deutschland
München, Deutschland
Paris, France
Bremen, Deutschland
Hlavní město Praha, Czechia
Hlavní město Praha, Czechia
Hlavní město Praha, Czechia
Frankfurt am Main, Deutschland
Veldhoven, Netherlands
Cedar Falls, IA
London, UK
London, UK
Wien, Österreich
Amsterdam, Netherlands
Bengaluru, India
Berlin, Germany
Berlin, Germany
Genève, Switzerland
Bunkyo City, Japan
Scrapping page 118
Chiyoda City, Japan
Shinagawa City, Japan
Minato C

Krefeld, Germany
Hyderabad, India
Hyderabad, India
San Diego, CA
Oak Ridge, TN
Panchkula, India
Osaka, Japan
Shibuya City, Japan
Shibuya City, Japan
Chuo City, Japan
横浜市, Japan
Shibuya City, Japan
Shinjuku City, Japan
Numazu, Japan
West Perth, Australia
Ueda, Japan
Ueda, Japan
Hyderabad, India
Hyderabad, India
Mumbai, India
Mumbai, India
Hyderabad, India
Hyderabad, India
Chandigarh, India
Kuala Lumpur, Malaysia
Scrapping page 136
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Scrapping page 137
Barcelona, Spain
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Bengaluru, India
Ben

Hyderabad, India
Hyderabad, India
Hyderabad, India
Minato City, Japan
Chiyoda City, Japan
Bunkyo City, Japan
Shibuya City, Japan
Chiyoda City, Japan
Shibuya City, Japan
Shibuya City, Japan
Chiyoda City, Japan
Chiyoda City, Japan
Toshima City, Japan
Minato City, Japan
Shibuya City, Japan
Shinagawa City, Japan
Shibuya City, Japan
Chiyoda City, Japan
Chiyoda City, Japan
Chiyoda City, Japan
Chiyoda City, Japan
Chiyoda City, Japan
Minato City, Japan
Chiyoda City, Japan
Nagoya, Japan
Scrapping page 155
Osaka, Japan
Shibuya City, Japan
Shinagawa City, Japan
Yokohama, Japan
Meguro City, Japan
Shinagawa City, Japan
Shinagawa City, Japan
Chiyoda City, Japan
Yokohama, Japan
Suita, Japan
Chiyoda City, Japan
Shinjuku City, Japan
Minato City, Japan
Minato City, Japan
Minato City, Japan
Minato City, Japan
Kyoto, Japan
福岡市, Japan
Shinjuku City, Japan
Hyderabad, India
Hyderabad, India
Pune, India
Hyderabad, India
Pune, India
Bengaluru, India
Scrapping page 156
Consolação, Brazil
Bengaluru, India
Mumbai

[]

## 2.13. StackOverflow Finish
* 지원 링크 추가 

In [71]:
# module import
import requests
from bs4 import BeautifulSoup

# 변수
URL = f"https://stackoverflow.com/jobs?q=python&sort=i"

# so.py
def get_last_page():
    req = requests.get(URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    pages = soup.find("div", {"class" : "s-pagination"}).find_all("a")
    last_page = pages[-2].get_text(strip=True) # page[-1] : 마지막 Next, page[-2] : 그 이전 마지막 페이지
    return int(last_page)

def extract_job(html): 
    title = html.find("div", {"class" :"grid--cell fl1"}).find("h2").find("a")["title"]
    company, location = html.find("div", {"class" : "grid--cell fl1"}).find("h3").find_all("span", recursive=False)
    company = company.get_text(strip=True)
    location = location.get_text(strip=True).strip("-")
    job_id = html["data-jobid"]
    link = f"https://stackoverflow.com/jobs/{job_id}"
    return {'title' : title, 'company' : company, 'location' : location, 'link' : link}

def extract_jobs(last_page):
    jobs = []
    for page in range(last_page):
        print(f"Scrapping page {page+1}")
        req = requests.get(f"{URL}&pg={page+1}")
        soup = BeautifulSoup(req.text, 'lxml')
        results = soup.find_all("div", {"class":"-job"})
        for result in results:
            job = extract_job(result) # 위의 함수로 result 넘김.
            jobs.append(job)   
    return jobs

def get_jobs():
    last_page = get_last_page()
    jobs = extract_jobs(last_page)
    return jobs


# main.py
get_jobs()

Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5
Scrapping page 6
Scrapping page 7
Scrapping page 8
Scrapping page 9
Scrapping page 10
Scrapping page 11
Scrapping page 12
Scrapping page 13
Scrapping page 14
Scrapping page 15
Scrapping page 16
Scrapping page 17
Scrapping page 18
Scrapping page 19
Scrapping page 20
Scrapping page 21
Scrapping page 22
Scrapping page 23
Scrapping page 24
Scrapping page 25
Scrapping page 26
Scrapping page 27
Scrapping page 28
Scrapping page 29
Scrapping page 30
Scrapping page 31
Scrapping page 32
Scrapping page 33
Scrapping page 34
Scrapping page 35
Scrapping page 36
Scrapping page 37
Scrapping page 38
Scrapping page 39
Scrapping page 40
Scrapping page 41
Scrapping page 42
Scrapping page 43
Scrapping page 44
Scrapping page 45
Scrapping page 46
Scrapping page 47
Scrapping page 48
Scrapping page 49
Scrapping page 50
Scrapping page 51
Scrapping page 52
Scrapping page 53
Scrapping page 54
Scrapping page 55
Scrapping page 56
S

[{'title': 'Software Engineer in Test',
  'company': 'READY Robotics',
  'location': 'Columbus, OH',
  'link': 'https://stackoverflow.com/jobs/357313'},
 {'title': 'Big Data Analytics Engineer',
  'company': 'Federal Hill Consulting',
  'location': 'Rockville, MD',
  'link': 'https://stackoverflow.com/jobs/284178'},
 {'title': 'Lead Backend Developer',
  'company': 'PriceSenz',
  'location': 'Los Angeles, CA',
  'link': 'https://stackoverflow.com/jobs/370188'},
 {'title': 'Fullstack Engineer - JavaScript (m/f/d)',
  'company': 'FINEWAY',
  'location': 'Munich, Germany',
  'link': 'https://stackoverflow.com/jobs/333805'},
 {'title': 'Software Engineer, Full Stack',
  'company': 'Facebook',
  'location': 'Tel Aviv-Yafo, Israel',
  'link': 'https://stackoverflow.com/jobs/223734'},
 {'title': 'Data Engineer, Machine Learning',
  'company': 'Facebook',
  'location': 'Fremont, CA',
  'link': 'https://stackoverflow.com/jobs/353943'},
 {'title': 'AR/VR Tools Engineer',
  'company': 'Facebook',

## 최종 정리
1. `extract_pagination` 함수
    * 두 사이트 모두 pagination 구조 비슷.
        * class명이 pagination인 div
        * anchor 태그
    * 만약 하나의 html 구조가 달라진다면 page 추출하는 것도 달라진다.(stackoverflow 경우 바뀌었다)

## 2.14~ csv 파일로 저장
* `w` : 쓰기 - 없으면 생성 + 이미 있는 파일이라도 덮어씀.
* writer 객체 생성 : 1csv.writer()
    - list로 작성하지 않고 string으로 작성하면, t,i,t,l,e ... 이런 식으로 저장됨.
* dict의 values를 list로 만들어서 저장.

In [80]:
# module import

import csv

# save.py

def save_to_file(jobs):
    file = open("jobs_StackOverflow.csv", mode="w", encoding="UTF-8") # 없으면 현재 경로에 파일 생성
    writer = csv.writer(file) # 방금 연 파일에 csv 작성
    writer.writerow(["title", "company", "location", "link"])
    for job in jobs:
        writer.writerow(list(job.values()))
    file.close()
    return

jobs_so = get_jobs()
save_to_file(jobs_so)

Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5
Scrapping page 6
Scrapping page 7
Scrapping page 8
Scrapping page 9
Scrapping page 10
Scrapping page 11
Scrapping page 12
Scrapping page 13
Scrapping page 14
Scrapping page 15
Scrapping page 16
Scrapping page 17
Scrapping page 18
Scrapping page 19
Scrapping page 20
Scrapping page 21
Scrapping page 22
Scrapping page 23
Scrapping page 24
Scrapping page 25
Scrapping page 26
Scrapping page 27
Scrapping page 28
Scrapping page 29
Scrapping page 30
Scrapping page 31
Scrapping page 32
Scrapping page 33
Scrapping page 34
Scrapping page 35
Scrapping page 36
Scrapping page 37
Scrapping page 38
Scrapping page 39
Scrapping page 40
Scrapping page 41
Scrapping page 42
Scrapping page 43
Scrapping page 44
Scrapping page 45
Scrapping page 46
Scrapping page 47
Scrapping page 48
Scrapping page 49
Scrapping page 50
Scrapping page 51
Scrapping page 52
Scrapping page 53
Scrapping page 54
Scrapping page 55
Scrapping page 56
S

### 최종 `main.py`
```
from indeed import get_jobs as get_indeed_jobs
from so import get_jobs as get_so_jobs
from save import save_to_file

indeed_jobs = get_indeed_jobs()
so_jobs = get_so_jobs()

jobs = indeed_jobs + so_jobs
save_to_file(jobs)
```
